<a href="https://colab.research.google.com/github/Falguni307/DataScience/blob/master/Movie_Recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  #Installing lightfm library
!pip install lightfm nmslib

In [ ]:
#Downloading data
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

In [ ]:
#Unarchiving data
!unzip ml-latest-small.zip

#Importing modules from the library

In [ ]:
from lightfm import LightFM
import nmslib
import pandas as pd
from scipy.sparse import coo_matrix

# Secondary functions

In [ ]:
#These are secondary functions  that transform data to a more suitable format
def update_data(movies,ratings):
  #function for tranforming the data
  ratings['ID'] = ratings.movieId.astype('category').cat.rename_categories(range(1, ratings.movieId.nunique()+1)).astype(int)
  movies = pd.merge(movies,ratings,how='inner',on='movieId').drop_duplicates('movieId')
  ratings.rating = ratings.rating.apply(lambda x: 1 if x> 4 else 0)
  ratings_sparse = coo_matrix((ratings.rating, (ratings.userId,ratings.ID)))
  return movies[['title','genres','ID']].set_index('ID'),ratings[['userId','ID','rating']],ratings_sparse

In [ ]:
# Function that works with the search
def init_nms (item_embeddings):
  # initialize a new nmslib index, using a HNSW index on Cosine Similarity
  nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
  nms_idx.addDataPointBatch(item_embeddings)
  nms_idx.createIndex(print_progress=True)
  return nms_idx

# Importing data


In [ ]:
movies =  pd.read_csv('ml-latest-small/movies.csv')
ratings =  pd.read_csv('ml-latest-small/ratings.csv')
#Transforming data
movies,ratings,ratings_sparse = update_data(movies,ratings)

# In order to display the data, you need to write the name of the table and then click play



In [ ]:
# To display a table, you need to write its name - movies
# Your code here:

## **Write the code to display first 5 movies**

In [ ]:
# In order to display first 5 elements ues function head 
# Your code here:


## Write code to dispaly ratings

In [ ]:
# Ratings are stored in a table ratings, in order to dispaly first elements use funcion head
# Your code here:


# Exploring movies

In [ ]:
# Nothing to write here,
# Let's look at the code
#@title Choose a film
movie_name = "Matrix" #@param {type:"string"}
movies[movies['title'].str.find(movie_name)>=0]

# Time for some magic! Let's create a model




In [ ]:
# Let's create a model LightFM with arguments (loss=LOSS, no_components=NO_COMPONENTS)
LOSS ="warp" #  Learning algorithm
NO_COMPONENTS = 30 # Dimension of the space
# Write your code here
model =  # Creating your model here, write your code here model = type of model (arguments)

# Training the model

In [ ]:
# Training the model, we need to call .fit function from our model with arguments (data,epochs) 
# Our data is user ratings stored in ratings_sparse
data = ratings_sparse 
# training the model, write your code here model.fit(...)

# Using the model

In [ ]:
# Here we got a new representation of our data
_, item_embeddings = model.get_item_representations()
# Building search
nms_idx = init_nms(item_embeddings)

# Secondary function


In [ ]:
# Secondary option for searching nearest movies
def nearest_movies_nms(movie_id, index, n=10):
    # Returns nearest movies
    nn = index.knnQuery(item_embeddings[movie_id], k=n)
    return nn

# Choosing a film for recommendation

In [ ]:
#@title Chosing a film
movie_name = "Inception" #@param {type:"string"}
ID = 7356 #@param {type:"number"}
movies[movies['title'].str.find(movie_name)>=0]

# Getting recommendations




In [ ]:
# Getting recommendation
rec = nearest_movies_nms(ID,nms_idx)[0]

# Displaying recommendations

In [ ]:
# Let's use table movies and function .loc with arguments [rec], 
# We need to use square brackets here
# Your code here

# We've got mediocre results, lt's improve them!

##  How can we improve our model?


In [ ]:
#@title Let's choose number of epochs for training
# We train the model, call .fit function of or model with arguments (data,epochs) 
epochs =  100#@param {type:"number"}
model.fit(data,epochs=epochs) # training the model

In [ ]:
import random
random.seed(42)

In [ ]:
# We've got a new representation of our data here
_, item_embeddings = model.get_item_representations()
# Let's build search
nms_idx = init_nms(item_embeddings)

In [ ]:
# Getting recommendations
rec = nearest_movies_nms(ID,nms_idx)[0]

#  What if change dimension from 30 to 1


In [ ]:
NO_COMPONENTS =   100#@param {type:"number"}
model = LightFM(loss=LOSS, no_components = NO_COMPONENTS) # Creating model here, write your code here
epochs =  100#@param {type:"number"}
model.fit(data,epochs=epochs) # Training the model, write your code here

In [ ]:
# Here we get a new representation of our data
_, item_embeddings = model.get_item_representations()
# Building search
nms_idx = init_nms(item_embeddings)

In [ ]:
# Getting recommendations
rec = nearest_movies_nms(ID,nms_idx)[0]

In [ ]:
nearest_movies_nms(ID,nms_idx)[1]

In [ ]:
movies.loc[rec]

What can be done?


1.   Change dimension parameters
2.   Change evaluation criterion

